# Motivation

This notebook is a demonstration on how to use some utility functions developed as part of the [Nern, A., Loesche, F., Takemura, Sy. et al. (2025) paper](https://doi.org/10.1038/s41586-025-08746-0). Here we refer to the code from the paper as **MA**le **D**rosophila **VI**sual **S**ystem **C**onnectome, abbreviated as `madvisc`.

The `madvisc` code was developed for the "Optic-Lobe" dataset, at the time of release supporting `optic-lobe:v1.0.1` and `optic-lobe:v1.1`. Both datasets are available at [neuPrint](https://neuprint.janelia.org/) with the code available at the [Reiser Lab GitHub repository](https://github.com/reiserlab/male-drosophila-visual-system-connectome-code).

While most of the code is specific to generating the figures in the paper, some of the utility functions provide easy access to properties such as the spatial location of neurons and synapses or even plotting functions to visualize the connectome. So we decided to expose the functions inside the `src/utils` and `src/queries` directories as a python package. 

Here we show a simple way to get started with the madvisc package. It can serve as a template for your own notebooks. For example, you can create a clone of this repository and then add your own notebooks in the `notebooks` directory.


# Installation

This package is currently only available on GitHub. While there might be other ways to install it, we recommend using **pixi** as we did for the original code. [Pixi](https://pixi.sh) is a package manager for Python that creates reproducible virtual environment for each project. We had good experiences with pixi for the original code developing it across Linux, macOS and Windows at the same time.

## Install Pixi

To install pixi itself, follow the instructions on the [pixi website](https://pixi.sh/latest/installation/). Follow the instructions for your operating system, in most cases executing one command line in your terminal will suffice.

Verify that pixi is installed by running `pixi --version` in your terminal. If you see a version number, you are good to go.

## Install this package

To install this package, you have to clone it from GitHub. It doesn't matter if you want to use the exact same repository or if you clone, fork, or copy it first. Once you have the repository on your local machine, you should open a terminal inside the repository directory.

Inside the repository directory, you can run the following command to install the dependencies and the package itself: `pixi install`. The first time you run this command, it will take a while to download and install all the depenencies. Alternatively, the command `pixi shell` will download all dependencies just like `pixi install` and open a shell inside the virtual environment.

If the installation was successful and you set the correct python kernel in your jupyter notebook, the following code should run without errors:

In [ ]:
from madvisc.utils import olc_client



## Configure the package

The package `madvisc` expects some configuration to be set up. Access to the data sets is provided by environment variables set up in a `.env` file in the project's root directory. The `.env` file is also used to find additional parameters from the `params` folder. To create the `.env` file, copy the content of `.env.sample` to `.env` and edit the values accordingly. To get started, it should be sufficient to copy the access toke from your [neuPrint account](https://neuprint.janelia.org/account) to the `NEUPRINT_APPLICATION_CREDENTIALS` variable in the `.env` file.

If the configuration is set up correctly, the following code should run without errors and print out some basic information about the dataset and client.

In [ ]:
c = olc_client.connect(verbose=True)

# Sample Code

The following code imports the class `OLNeuron` from the `madvisc` package.

We initialize the `OLNeuron` class with the ID of an existing neuron, for example `104599`. The code then prints the type of the neuron and a table of the main columns this neuron is associated with.

In [ ]:
from madvisc.utils.ol_neuron import OLNeuron

# Tm5a: 16354 ; C2: 86459; TmY5a: 104599
oln = OLNeuron(104599)
print(f"The neuron is of type '{oln.get_type()}'")
print(f"Hex IDs:\n{oln.get_hex_id()}")
print(f"All innervated ROIs:\n{oln.innervated_rois()}")

Another useful class is `NeuronBag`, which represents a collection of neurons. Initializing the class with the name of a neuron type such as `Tm20` and the `side` set to `R-dominant` will return all neurons of that type associated with the right side of the optic lobe. The NeuronBag class also provides a method to order the neurons by their spatial distance to a given column. Here we sort the whole bag by distance to the arbitrarily chosen column (13, 10) and I print the first 5 neurons and their columns via the `OLNeuron` class. __Note:__ it can take a few minutes to sort all neurons by distance.

In [ ]:
from madvisc.utils.neuron_bag import NeuronBag

bag = NeuronBag('Tm20')
bag.sort_by_distance_to_hex(neuropil='ME(R)', hex1_id=13, hex2_id=10)

closest = bag.get_body_ids(5)
for idx, next_neuron in enumerate(closest):
    # print(f"{idx} {next_neuron}")
    tm20_dist = OLNeuron(next_neuron)
    print(f"{idx} {tm20_dist.get_hex_id()}\n")

# Advanced usage

## Modify parameters

The `madvisc` package uses some parameter files inside the `params` directory. When using `madvisc`, these parameters themselves are not inherited, but depending on the funcitonality required, the location of these parameter files are. This makes it very easy to adapt the list like the `Primary_cell_type_table` to a new dataset: just provide your own parameters in the file `params/Primary_cell_type_table.xlsx` and the code will use these parameters. If parameter files are not required for your specific use case, you should modify the classes instead (simiarl approach to the added functionality below).

## Extend funcitonality 

The following code example extends functionality of an existing class from `madvisc`. Specifically, we extend the `OLNeuron` class with a method to get all the layers it innervates. 

Here the new class `TemplateNeuron` (uninspiredly named after the madvisc-template repository, please find better names in your code!) inherits everything from `OLNeuron`. This is demonstrated by the call to `tn.get_type()`.

`TemplateNeuron` also possesses an additional method called `get_layer_ids()` which returns all ROIs that follow a specific name structure containing `_layer_` in a slightly different format, specifically a dictionary with layer names as keys and the layer IDs as a list of numbers. __Note:__ This doesn't really make too much sense in life code, instead you would probably want to query the database for the synapses and their location within an ROI, but this is just a demonstration how to extgend the functionality of an existing class.

Inherited classes should not be defined inside a notebook but rather in a separate file inside the `src` directory. That way you can adapt the classes for data sets other than the optic lobe and reuse them across notebooks.

In [ ]:
import re

class TemplateNeuron(OLNeuron):

    """
    An example class to inherit from OLNeuron and provide an additional method.
    """

    def get_layer_ids(self):
        """
        Get the innervated layers in the optic lobe from a list of named ROIs.

        Returns
        -------
        dict
            A dictionary where keys are layer names (e.g. 'ME (R)' for right medulla) 
            and the values are a list of layer numbers.
        """
        rois = self.innervated_rois()
        pattern = re.compile(r'(.*?)_([RL])_layer_(\d+)', re.IGNORECASE)
        layers = {}
        for roi in rois:
            for layer in roi:
                if match := pattern.search(layer):
                    key = f"{match.group(1)} ({match.group(2)})"
                    if key in layers:
                        layers[key].append(int(match.group(3)))
                    else:
                        layers[key] = [int(match.group(3))]
        return layers

        

tn = TemplateNeuron(104599)
print(f"Once again the type: {tn.get_type()}")
tn.get_layer_ids()
